In [1]:
from collections import defaultdict
import random

import mip
import json
import networkx as nx

ModuleNotFoundError: No module named 'mip'

In [3]:
def multicommodity_flow(g, h, u, b):
    
    model = mip.Model()
    
    flow = {(f, e): model.add_var(f"flow-{f}-{e}", lb=0, obj=1) for e in set(g.edges()) for f in h.edges()}

    for e in g.edges():
        model.add_constr(mip.quicksum(flow[f, e] for f in h.edges()) <= u[e])

    for f in h.edges():
        (s, t) = f
        
        for v in g.nodes():
            model.add_constr(
                  mip.quicksum(flow[f, e] for e in set(g.out_edges(v)))
                - mip.quicksum(flow[f, e] for e in set(g.in_edges(v)))
                == (1 if v == s else 0) * b[f] + (-1 if v == t else 0) * b[f])
    
    model.optimize()
    
    
    
    return model.status, {k: v.x for k, v in flow.items() if v.x is not None and v.x > 0}
